<a href="https://colab.research.google.com/github/sunnysavita10/Complete-LLM-Finetuning/blob/main/Knowledge_DIstillation_in_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classical LLM Distillation (BERT → DistilBERT)

> **Goal:** Train a small **student** model to learn from a large **teacher** model by combining *soft* predictions (teacher outputs) with *hard* labels (ground truth).

## 📚 References
- [Hinton et al., *Distilling the Knowledge in a Neural Network* (2015)](https://arxiv.org/pdf/1503.02531)  
- [*Knowledge Distillation: A Survey*](https://arxiv.org/pdf/2006.05525)  
- [*DistilBERT: smaller, faster, cheaper and lighter*](https://arxiv.org/pdf/1910.01108)  
- [*TinyStories: How Small Can Language Models Be and Still Speak Coherent English?*](https://arxiv.org/pdf/2305.07759)  
- [*Distilling Step-by-Step! Outperforming Larger Language Models with Less Training Data and Smaller Model Sizes*](https://arxiv.org/pdf/2305.02301)  

---

## Overview
This is a step-by-step outline of how Large Language Model (LLM) distillation is performed, using **BERT** as the teacher model and **DistilBERT** as the student model.

---

## Process Breakdown

| Block                   | Purpose                                                                                                                              |
| ----------------------- | ------------------------------------------------------------------------------------------------------------------------------------ |
| **0. Imports & Config** | Import all required Python libraries (Transformers, Torch, etc.), detect GPU/CPU, and set hyperparameters (batch size, learning rate, epochs, temperature, alpha). |
| **1. Dataset**          | Load a demo text dataset, tokenize it using the tokenizer; the data collator automatically pads sequences so each batch is uniform. |
| **2. Models**           | Load pretrained **BERT** as the teacher (freeze parameters), and **DistilBERT** as the student (trainable).                         |
| **3. Losses**           | Use two loss functions: <br>• **CrossEntropy (CE)** = for hard labels (ground truth) <br>• **KL Divergence** = for soft labels (teacher’s logits → softmax with temperature). |
| **4. Optimizer**        | Use **AdamW** optimizer with a **linear learning rate scheduler** for smoother and more stable training.                            |
| **5. distill_epoch()**  | For each batch: <br>• Get teacher logits and create soft targets with temperature <br>• Get student logits <br>• Compute soft loss and hard loss, combine them using α <br>• Backpropagate **only** through the student model. |
| **evaluate()**          | Evaluate the student model’s accuracy on the validation set to monitor performance improvements.                                   |
| **Loop**                | For each epoch: run `distill_epoch()` followed by `evaluate()`.                                                                     |
| **Save**                | Save the fine-tuned student model to disk for future inference.                                                                     |

---

# Distillation: step-by-step (concise reference)

> **Tip:** For very large datasets or multi-GPU/TPU training use Hugging Face’s `DistillationTrainer` or `accelerate`. The core algorithm below stays the same.

---

## Overview (what’s happening)
1. Train a large **teacher** model (high capacity) normally and freeze it.  
2. Train a smaller **student** model to mimic the teacher **and** the ground-truth labels.  
3. Student loss = weighted combination of a **soft** (teacher) loss and a **hard** (label) loss.

---

## Step-by-step

### 1. Teacher output (`t_soft`)
- Compute teacher logits: `z_T = teacher(x)` (teacher is frozen; run under `torch.no_grad()`).
- Apply **temperature** `T` and softmax to get *soft targets*:
  \[
  p_T = \text{softmax}\!\left(\frac{z_T}{T}\right)
  \]
- `T > 1` “softens” the distribution (reveals class similarities).

### 2. Student output (`s_soft`)
- Compute student logits: `z_S = student(x)`.
- Convert to log-probabilities at the same temperature:
  \[
  \log q_S = \log\text{softmax}\!\left(\frac{z_S}{T}\right)
  \]

### 3. Distillation (soft) loss
- Use KL divergence (teacher distribution → student distribution).
- In PyTorch style: `nn.KLDivLoss(reduction='batchmean')(log_q_S, p_T)`
- Multiply the KL loss by `T^2` to correct gradient scale (Hinton et al.):
  \[
  L_{\text{soft}} = T^2 \cdot \text{KL}(p_T \,\|\, q_S)
  \]

### 4. Hard (label) loss
- Standard cross-entropy between student logits and true labels:
  \[
  L_{\text{hard}} = \text{CE}(z_S, y)
  \]

### 5. Combine
- Weighted sum:
  \[
  L = \alpha \cdot L_{\text{soft}} + (1-\alpha)\cdot L_{\text{hard}}
  \]
- Typical choices: `T ∈ [2,5]`, `α ≈ 0.5` (tune for your task).
- If `α = 1` → pure distillation (no hard labels). If `α = 0` → normal fine-tuning (no distillation).

---

## Implementation notes / best practices
- Freeze teacher: `teacher.eval()` and use `with torch.no_grad()` when generating `z_T`. This saves memory and avoids updating teacher weights.
- Use `F.softmax(z_T / T, dim=-1)` for teacher targets and `F.log_softmax(z_S / T, dim=-1)` for student input to `KLDivLoss`.
- In PyTorch, prefer `nn.KLDivLoss(reduction='batchmean')` for stable gradients.
- Multiply KL term by `T**2` (important — otherwise gradients from soft targets are scaled down).
- Monitor both components (`L_soft`, `L_hard`) and validation accuracy.

---

## Small numeric example (temperature effect)
- Teacher logits: `[10, 2]`
  - `T = 1` → `softmax([10,2]) ≈ [0.9997, 0.0003]` (very peaked)
  - `T = 2` → `softmax([5,1]) ≈ [0.982, 0.018]` (softer, reveals second choice)
  - `T = 5` → `softmax([2,0.4]) ≈ [0.83, 0.17]` (much softer)
- Softer distributions reveal the teacher’s relative beliefs and help the student learn nuanced class relations.

---

## Why distillation helps
- **Soft targets** encode “dark knowledge”: relative similarities between classes that hard labels hide.  
- Student learns both the dataset labels **and** the teacher’s nuanced behavior → better generalization for a much smaller model.

---

## Short pseudocode (conceptual)
1. `z_T = teacher(x)`  (no grad)
2. `p_T = softmax(z_T / T)`
3. `z_S = student(x)`
4. `log_q_S = log_softmax(z_S / T)`
5. `loss_soft = T^2 * KLDiv(log_q_S, p_T)`
6. `loss_hard = CrossEntropy(z_S, y)`
7. `loss = alpha * loss_soft + (1 - alpha) * loss_hard`
8. `loss.backward()` and `optimizer.step()` (update only student)

---

## Quick hyperparameter suggestions
- `T = 2` (good starting point), try `2–5`.  
- `alpha = 0.3–0.7` depending on trust in teacher vs labels.  
- `batch size`: 64–256 (task-dependent).  
- Ensure teacher has good accuracy before distillation.

---

## Final note
Distillation is **not** just fine-tuning: it explicitly transfers the teacher’s learned distributional knowledge (soft targets) into a compact student while still respecting hard labels. For large-scale runs, use `DistillationTrainer` / `accelerate` to scale cleanly across devices.


In [ ]:
# !pip install --upgrade datasets fsspec transformers

  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

In [4]:
batch_size   = 16
lr           = 5e-5
epochs       = 1
temperature  = 2.0
alpha_soft   = 0.5
max_len      = 128
device       = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Load dataset
raw = load_dataset("tweet_eval", "sentiment")

In [6]:
label_feature = raw["train"].features["label"]

In [7]:
print("Label names:", label_feature.names)

Label names: ['negative', 'neutral', 'positive']


In [8]:
# Subset (2.5k samples for train)
train = raw['train'].shuffle(seed=42).select(range(2500))

Why Train is Subset, but Val is Not

🔹 1. Training cost is high, validation cost is low

Training = multiple forward + backward passes → GPU heavy

Validation = only forward pass, no gradient update → fast

So it’s common to reduce training size for quick experiments but keep full validation for accurate metric evaluation.

🔹 2. Keeping val full improves generalization check

If you also reduce validation (e.g., from 872 → 100), metrics become noisy and unreliable.

Full validation gives stable accuracy/loss during training.

In [9]:
val   = raw['validation']

In [10]:
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [11]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, max_length=max_len)

In [12]:
# Tokenize & remove original text column
tokenized = {}

In [13]:
tokenized['train'] = train.map(tokenize, batched=True, remove_columns=['text'])
tokenized['validation'] = val.map(tokenize, batched=True, remove_columns=['text'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
# Data Collator (auto-padding)
collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [15]:
# DataLoaders
train_dl = DataLoader(tokenized['train'], batch_size=batch_size,shuffle=True, collate_fn=collator)

In [16]:
val_dl = DataLoader(tokenized['validation'], batch_size=batch_size,shuffle=False, collate_fn=collator)

In [17]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [18]:
num_labels = 3

In [19]:
teacher = AutoModelForSequenceClassification.from_pretrained(
    "bert-large-uncased", num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
student = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Freeze teacher (no training): We “lock” the teacher so it doesn’t learn anymore and ensure it behaves predictably while 
# generating soft targets for the student
for p in teacher.parameters():
    p.requires_grad = False
teacher.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [22]:
ce_loss = nn.CrossEntropyLoss()

In [23]:
kl_loss = nn.KLDivLoss(reduction="batchmean")

In [24]:
optimizer = optim.AdamW(student.parameters(), lr=lr)

In [25]:
from transformers import get_scheduler

In [ ]:
# Create a learning rate scheduler that linearly decreases the learning rate from
# the initial value to zero over the course of training.
lr_scheduler = get_scheduler(
    name="linear",                  # Scheduler type: linear decay
    optimizer=optimizer,            # Optimizer whose LR will be scheduled
    num_warmup_steps=0,             # No warmup steps, LR starts decaying immediately
    num_training_steps=len(train_dl) * epochs  # Total steps = batches per epoch * number of epochs
)

In [27]:
from tqdm.auto import tqdm

## Summary of `distill_epoch()` Function

The `distill_epoch()` function runs one training epoch of **knowledge distillation**, where a smaller **student model** learns from a larger, pretrained **teacher model** by combining teacher outputs with true labels.

- **Student in training mode:** Enables dropout and training behaviors.
- **Batch processing:** For each batch:
  - Move inputs (`input_ids`, `attention_mask`, `labels`) to the device (CPU/GPU).
  - **Teacher forward pass (no grad):** Compute teacher logits, then generate *soft targets* with temperature-scaled softmax.
  - **Student forward pass:** Compute student logits and log-softmax with the same temperature.
  - **Loss computation:**
    - *Soft loss:* KL divergence between student and teacher output distributions, scaled by temperature squared.
    - *Hard loss:* Cross-entropy loss between student logits and ground-truth labels.
    - Combine losses with weighting factor `alpha_soft`.
  - **Optimization:**
    - Backpropagate combined loss.
    - Update student parameters via optimizer.
    - Adjust learning rate via scheduler.
- **Progress bar:** Displays training loss dynamically.

This process helps the student model learn both the nuanced behavior of the teacher (via soft targets) and the actual labels (hard targets), resulting in a smaller, efficient model approximating the teacher’s performance.


In [ ]:
def distill_epoch():
    # Set student model to training mode (enable dropout, batch norm updates, etc.)
    student.train()
    
    # Wrap dataloader with tqdm for a progress bar display
    pbar = tqdm(train_dl, desc="Train")
    
    # Iterate over batches in the training dataloader
    for batch in pbar:
        # Move batch data to the right device (GPU or CPU)
        input_ids = batch["input_ids"].to(device)
        attention = batch["attention_mask"].to(device)
        labels    = batch["labels"].to(device)

        # ---------------- Teacher forward pass ---------------- #
        # Disable gradient calculation for teacher to save memory and computation
        with torch.no_grad():
            # Get raw logits (pre-softmax scores) from teacher model
            t_logits = teacher(input_ids, attention_mask=attention).logits
            # Apply softmax with temperature to get soft probability targets
            t_soft = torch.softmax(t_logits / temperature, dim=1)

        # ---------------- Student forward pass ---------------- #
        # Get raw logits from student model (these will be updated via backprop)
        s_logits = student(input_ids, attention_mask=attention).logits
        # Apply log softmax with same temperature for KL divergence loss calculation
        s_soft = torch.log_softmax(s_logits / temperature, dim=1)

        # ---------------- Loss calculation ---------------- #
        # Calculate soft loss: KL divergence between student and teacher distributions
        # Multiply by temperature^2 to properly scale gradients (Hinton et al.)
        loss_soft = kl_loss(s_soft, t_soft) * (temperature ** 2)
        
        # Calculate hard loss: cross-entropy between student logits and true labels
        loss_hard = ce_loss(s_logits, labels)
        
        # Combine losses with weighting factor alpha_soft
        # alpha_soft controls importance of soft vs hard loss components
        loss = alpha_soft * loss_soft + (1 - alpha_soft) * loss_hard

        # ---------------- Backpropagation & optimization ---------------- #
        # Zero gradients from previous step
        optimizer.zero_grad()
        # Backpropagate combined loss through student network only
        loss.backward()
        # Update student model parameters
        optimizer.step()
        # Update learning rate scheduler if applicable
        lr_scheduler.step()

        # Update progress bar with current loss value
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})


## `evaluate()` Function Explanation

This function evaluates the **student** model’s accuracy on the validation dataset.

- **Set model to evaluation mode:**  
  `student.eval()` disables dropout and other training-specific layers for stable inference.

- **Initialize counters:**  
  `correct` counts correctly predicted samples, and `total` counts total samples processed.

- **No gradient calculation:**  
  `with torch.no_grad()` reduces memory usage and speeds up inference since no backpropagation is needed.

- **Batch-wise evaluation loop:**  
  For each batch in the validation dataloader (`val_dl`):  
  - Move inputs (`input_ids`, `attention_mask`, `labels`) to the appropriate device (CPU/GPU).  
  - Forward pass through the student model to get raw logits.  
  - Compute predicted classes by taking the `argmax` over logits.  
  - Compare predictions with ground-truth labels and update the count of correct predictions.  
  - Increment total samples processed.

- **Calculate accuracy:**  
  Returns the accuracy as a percentage, rounded to two decimal places:  
  \[
  \text{Accuracy} = \frac{\text{correct}}{\text{total}} \times 100
  \]

This function helps monitor how well the student model performs on unseen data during or after training.


In [ ]:
def evaluate():
    # Set student model to evaluation mode (disables dropout, batchnorm, etc.)
    student.eval()
    
    correct = 0  # Counter for correct predictions
    total = 0    # Counter for total samples processed
    
    # Disable gradient calculations for faster inference and lower memory usage
    with torch.no_grad():
        # Loop over batches in the validation dataloader
        for batch in val_dl:
            # Move input ids, attention masks, and labels to the device (GPU or CPU)
            ids  = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            lbl  = batch["labels"].to(device)
            
            # Forward pass: get the logits output by the student model
            out = student(ids, attention_mask=attn).logits
            
            # Get predicted class by taking the index with highest logit value
            pred = out.argmax(dim=1)
            
            # Count how many predictions matched the true labels
            correct += (pred == lbl).sum().item()
            
            # Keep track of total number of samples processed
            total += lbl.size(0)
    
    # Calculate and return accuracy percentage rounded to 2 decimals
    return round(correct / total * 100, 2)


In [30]:
for ep in range(1, epochs + 1):
    distill_epoch()
    acc = evaluate()
    print(f"Epoch {ep}/{epochs} | Validation Accuracy: {acc}%")

Train:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 1/1 | Validation Accuracy: 62.7%


In [31]:
# ---------- 6. Save Student ----------
student.save_pretrained("distilled_student_model")
tokenizer.save_pretrained("distilled_student_model")

('distilled_student_model/tokenizer_config.json',
 'distilled_student_model/special_tokens_map.json',
 'distilled_student_model/vocab.txt',
 'distilled_student_model/added_tokens.json',
 'distilled_student_model/tokenizer.json')

## Summary: `predict_and_evaluate` Function

- **Purpose:**  
  Evaluate a trained model's accuracy on a test dataset and measure inference time.

- **Key steps:**  
  1. Sets the model to evaluation mode (`model.eval()`) to disable training-specific layers like dropout.  
  2. Iterates over the test dataset without computing gradients for efficiency.  
  3. Collects predicted labels and true labels for all test samples.  
  4. Computes accuracy using `sklearn.metrics.accuracy_score`.  
  5. Measures total inference time and calculates average time per sample.  
  6. Prints and returns accuracy, total inference time, and average per-sample inference time.

- **Usage:**  
  Useful for benchmarking model performance and speed on unseen data.



In [ ]:
from sklearn.metrics import accuracy_score
import time

def predict_and_evaluate(model, name, test_dl):
    # Set the model to evaluation mode (disables dropout, batchnorm, etc.)
    model.eval()
    
    all_preds, all_labels = [], []  # Lists to store predictions and true labels
    start_time = time.time()        # Record start time to measure inference duration

    # Disable gradient calculations to speed up inference and save memory
    with torch.no_grad():
        # Loop through batches in the test dataloader
        for batch in test_dl:
            # Move inputs and labels to the correct device (CPU/GPU)
            ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            lbls = batch["labels"].to(device)

            # Forward pass: get model logits
            logits = model(ids, attention_mask=attn).logits
            
            # Get predicted class indices by taking the max logit along dim=1
            preds = torch.argmax(logits, dim=1)

            # Append predictions and true labels to the lists (move to CPU and convert to list)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(lbls.cpu().tolist())

    # Calculate total inference time
    total_time = time.time() - start_time
    
    # Calculate accuracy using sklearn's accuracy_score
    acc = accuracy_score(all_labels, all_preds)
    
    # Calculate average time per sample (total time divided by total samples)
    avg_time = total_time / len(test_dl.dataset)

    # Print summary metrics
    print(f"\n {name}")
    print(f" Accuracy: {acc*100:.2f}%")
    print(f" Total Inference Time: {total_time:.2f} sec")
    print(f" Avg Time per Sample: {avg_time:.4f} sec")
    
    # Return accuracy, total inference time, and average time per sample
    return acc, total_time, avg_time


In [33]:
# --------- Load test set ---------
test = load_dataset("tweet_eval", "sentiment", split="test[:500]")  # sample test
tokenized_test = test.map(tokenize, batched=True, remove_columns=["text"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dl = DataLoader(tokenized_test, batch_size=batch_size, shuffle=False, collate_fn=collator)

In [34]:
# --------- Compare Teacher vs Student ---------
predict_and_evaluate(teacher, name="TEACHER (BERT-Large)", test_dl=test_dl)
predict_and_evaluate(student, name="STUDENT (Distilled BERT)", test_dl=test_dl)



 TEACHER (BERT-Large)
 Accuracy: 22.00%
 Total Inference Time: 3.63 sec
 Avg Time per Sample: 0.0073 sec

 STUDENT (Distilled BERT)
 Accuracy: 60.80%
 Total Inference Time: 1.16 sec
 Avg Time per Sample: 0.0023 sec


(0.608, 1.1580901145935059, 0.0023161802291870115)

Backed by Research

📄 “Distilling Step-by-Step” (Google, ACL 2023)

A 770M T5 student outperformed PaLM-540B teacher on multiple tasks using rationale distillation.

📄 TinyBERT paper (Huawei, 2020)

Task-specific distillation allowed TinyBERT to beat BERT-base on SST-2 and MNLI.

| Reason                               | Explanation                                                            |
| ------------------------------------ | ---------------------------------------------------------------------- |
| **TweetEval = small, noisy data** | BERT-Large is overfitting or underconfident due to task size           |
| **Student is fine-tuned**         | You updated student weights on TweetEval task                          |
| **Teacher is frozen**             | You're using teacher just for soft logits, not re-finetuning           |
| **Teacher not task-specific**     | Your BERT-Large is general, but student is task-tuned via distillation |


| Model   | Accuracy | Speed | Comment                                                 |
| ------- | -------- | ----- | ------------------------------------------------------- |
| Teacher | 22%      | Slow  | Not tuned, generic, likely overfitting/underfitting     |
| Student | 60.8%    | Fast  | Task-specific distilled, learned from soft+hard targets |


## With LLM


Teacher (LLaMA‑2 7B) produces a probability distribution (next‑token probabilities) for each prompt.

Student (TinyLLaMA 1.1B) is trained to:

Mimic the teacher’s distribution (KLDivLoss → soft targets).

Be correct on the ground‑truth labels (CrossEntropyLoss → hard targets).

During each prompt, backpropagation updates only the Student’s weights.

As a result, the Student copies the Teacher’s reasoning behavior while being smaller and faster.

How is it run in practice?

The given snippet is just a mini example (3 prompts).

In real distillation:

Use millions of prompts (including synthetic ones).

Train in batches with Accelerate/DeepSpeed to manage GPU memory.

Precompute and store teacher outputs offline, then train the Student on those for a faster pipeline.

#### teacher_id = "meta-llama/Llama-2-7b-chat-hf"      # big model (teacher)
#### student_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"  # small model (student)

#### teacher_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#### student_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [21]:
!pip install transformers accelerate bitsandbytes

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim

In [23]:
teacher_id = "microsoft/phi-2"
student_id = "microsoft/phi-1_5"

In [24]:
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_id)
student_tokenizer = AutoTokenizer.from_pretrained(student_id)

In [25]:
if teacher_tokenizer.pad_token is None:
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
if student_tokenizer.pad_token is None:
    student_tokenizer.pad_token = student_tokenizer.eos_token

In [26]:
teacher = AutoModelForCausalLM.from_pretrained(
    teacher_id,
    device_map="auto",
    load_in_8bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
for p in teacher.parameters():
    print(p)

Parameter containing:
tensor([[ 1.6998e-02, -1.3275e-02,  2.0309e-02,  ...,  1.7822e-02,
          5.1346e-03, -7.7343e-04],
        [ 9.7351e-03,  5.1636e-02,  1.5656e-02,  ..., -6.7329e-03,
          6.9389e-03, -1.1322e-02],
        [-4.6387e-02, -9.0942e-03, -1.1349e-03,  ..., -3.0945e-02,
          3.8940e-02,  1.3847e-02],
        ...,
        [-5.9605e-08,  5.9605e-08, -5.9605e-08,  ...,  1.5140e-05,
         -1.1206e-05,  1.7762e-05],
        [-0.0000e+00, -5.9605e-08, -1.1921e-07,  ..., -2.5094e-05,
          3.7730e-05,  2.0683e-05],
        [ 0.0000e+00, -5.9605e-08,  5.9605e-08,  ..., -1.6034e-05,
         -1.5676e-05, -3.5882e-05]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
Parameter containing:
Parameter(Int8Params([[ 26,  -5, -23,  ..., -30,  15,  11],
            [-37, -16, -10,  ...,  15,   0, -26],
            [-22, -11,  -4,  ...,   0,   9,   9],
            ...,
            [-35, -26,   5,  ...,  20,  12,  34],
            [-37,  50,  -2,  ...

In [28]:
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

In [29]:
student = AutoModelForCausalLM.from_pretrained(
    student_id,
    device_map="auto",
    load_in_8bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [30]:
prompts = [
    "Explain why the sky is blue. ### The sky appears blue because molecules in Earth's atmosphere scatter sunlight, and blue light is scattered more than other colors due to its shorter wavelength.",
    "What is the capital of France? ### The capital of France is Paris.",
    "Write a short story about a robot and a cat. ### Once upon a time, a lonely robot found a stray cat. They became best friends, exploring the city together, and the robot learned the meaning of companionship."
]


In [31]:
# ----- Distillation Hyperparams -----
temperature = 2.0
alpha_soft  = 0.7
ce_loss     = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
kl_loss     = nn.KLDivLoss(reduction="batchmean")
optimizer   = optim.AdamW(student.parameters(), lr=2e-5)

In [33]:
for prompt in prompts:
    # --- Tokenize separately ---
    t_inputs = teacher_tokenizer(prompt, return_tensors="pt", padding=True).to(teacher.device)
    s_inputs = student_tokenizer(prompt, return_tensors="pt", padding=True).to(student.device)

    with torch.no_grad():
        t_logits = teacher(**t_inputs).logits[:, :-1, :]
        t_soft = torch.softmax(t_logits / temperature, dim=-1)
        t_soft = torch.clamp(t_soft, min=1e-8)  # 🛠️ avoid log(0) in KLDiv

    s_logits = student(**s_inputs).logits[:, :-1, :]
    s_log_soft = torch.log_softmax(s_logits / temperature, dim=-1)

    labels = s_inputs["input_ids"][:, 1:].contiguous()

    loss_hard = ce_loss(s_logits.reshape(-1, s_logits.size(-1)), labels.reshape(-1))
    loss_soft = kl_loss(s_log_soft, t_soft) * (temperature ** 2)

    loss = alpha_soft * loss_soft + (1 - alpha_soft) * loss_hard

    if torch.isnan(loss):
        print("⚠️ NaN detected on prompt:", prompt[:50])
        continue  # skip to next prompt

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Prompt: {prompt[:40]}..., Loss: {loss.item():.4f}")

Prompt: Explain why the sky is blue. ### The sky..., Loss: nan
Prompt: What is the capital of France? ### The c..., Loss: nan
Prompt: Write a short story about a robot and a ..., Loss: nan


In [ ]:
student.save_pretrained("distilled_phi1_5")
student_tokenizer.save_pretrained("distilled_phi1_5")
print("Distillation complete. Model saved in 'distilled_phi1_5/'")